# 吴恩达深度学习第一课 作业

## week-1

### 1.1 用numpy构建sigmoid函数

sigmoid函数
$$
\sigma(x) = \frac{1}{1+e^{-x}}
$$

In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

In [5]:
x = np.array([1, 2, 3])
sigmoid(x)

array([0.73105858, 0.88079708, 0.95257413])

### 1.2 构建sigmoid的导数(梯度)

$$
{\sigma}'(z) = 
\frac{-e^{-z}}{(1+e^{-z})^{2}} = 
\frac{1}{1 + e^{-z}}(1 - \frac{1}{1 + e^{-z}}) = 
\sigma(z)(1-\sigma(z))
$$

In [6]:
def sigmoid_grad(x):
    s = sigmoid(x)
    ds = s * (1 - s)
    return ds

In [7]:
x = np.array([1, 2, 3])
sigmoid_grad(x)

array([0.19661193, 0.10499359, 0.04517666])

### 1.3 np中的 shape 和 reshape 函数

X.shape用于获取矩阵/向量X的shape(维度)  
X.reshape(...) 用于将X重塑为其他尺寸

In [27]:
a = np.random.rand(3,3,2)
# a = np.random.rand(18, 1)

print("a = " + str(a))

print("a.shape = " + str(a.shape))

b = a.reshape(a.shape[0] * a.shape[1] * a.shape[2], 1)
# b = a.reshape(3 ,3, 2)

print("b = " + str(b))

print("b.shape = " + str(b.shape))

a = [[[0.53162315 0.52859771]
  [0.68759983 0.58952027]
  [0.36429866 0.96568101]]

 [[0.30466324 0.78140938]
  [0.97725038 0.27599292]
  [0.30757616 0.40457955]]

 [[0.77707515 0.75517586]
  [0.39268482 0.83819877]
  [0.85628051 0.28430032]]]
a.shape = (3, 3, 2)
b = [[0.53162315]
 [0.52859771]
 [0.68759983]
 [0.58952027]
 [0.36429866]
 [0.96568101]
 [0.30466324]
 [0.78140938]
 [0.97725038]
 [0.27599292]
 [0.30757616]
 [0.40457955]
 [0.77707515]
 [0.75517586]
 [0.39268482]
 [0.83819877]
 [0.85628051]
 [0.28430032]]
b.shape = (18, 1)


### 1.4 行标准化

对数据归一化后，梯度下降的收敛速度更快，效果更好

$$
x \to \frac{x}{\left \| x \right \| }
$$

$$
\| x\| = np.linalg.norm(x, axis = 1, keepdims = True)
$$

In [39]:
x = np.random.randint(low=0, high=9, size=(2, 3))
print("x.shape = " + str(x.shape))  # x.shape = (2, 3)
print("x = " + str(x))

x_norm = np.linalg.norm(x, axis=1, keepdims=True)
# axis=0：沿着列的方向执行操作，对每一列的元素进行聚合操作。
# axis=1：沿着行的方向执行操作，对每一行的元素进行聚合操作。
print("x_norm.shape = " + str(x_norm.shape))    # x_norm.shape = (2, 1)
print("x_norm = " + str(x_norm))

xx = x / x_norm
# x 和 x_norm 维度不一致
# x 和 x_norm 行数一致 x.shape = (2, 3)  x_norm.shape = (2, 1)
# np计算时自动将 x_norm 的列扩展补全(broadcast)
print("xx.shape = " + str(x.shape))  # xx.shape = (2, 3)
print("xx = " + str(xx))


x.shape = (2, 3)
x = [[1 1 8]
 [6 1 3]]
x_norm.shape = (2, 1)
x_norm = [[8.1240384 ]
 [6.78232998]]
xx.shape = (2, 3)
xx = [[0.12309149 0.12309149 0.98473193]
 [0.88465174 0.14744196 0.44232587]]


封装成函数

In [ ]:
def normalizeRows(x):    
    x_norm = np.linalg.norm(x, axis=1, keepdims=True)
    xx = x / x_norm
    return xx

np.multiply(A, B) 和 A * B (相当于matlab的 A .* B ) 即相同位置的相乘
np.dot(A, B) 是矩阵的向量乘
加减乘除同理

## week-2

### 2.1 重塑维度

一个对于一个图片训练集/测试集 train_x_set  
其shape为(m, px, py, 3)  
重塑其维度(展平)  
train_set_x_flatten = train_x_set.reshape(train_x_set.shape[0], -1).T  
//将 train_x_set 重塑为 (m, px * py * 3) 然后转置 得到 (px * py * 3, m)

### 2.2 标准化数据集

对图片 RGB值范围为[0, 255]  
标准化只需  
train = train_set_x_flatten / 255

### 2.3 建立神经网络

**步骤**  
1. 定义模型结构(输入特征数量)  
2. 初始化模型参数  
3. 循环训练 正向传播 计算当前损失 & 反向传播 计算当前梯度 & 梯度下降 更新参数


定义**传播函数**  
propagate(w, b, X, Y)  
实现前向和后向传播的传播函数，计算成本函数及其梯度。  

参数：  
 - w  - 权重，大小不等的数组（num_px * num_px * 3，1）  
 - b  - 偏差，一个标量  
 - X  - 矩阵类型为（num_px * num_px * 3，训练数量）  
 - Y  - 真正的“标签”矢量（如果非猫则为0，如果是猫则为1），矩阵维度为(1,训练数据数量)  

返回：  
 - cost - 逻辑回归的负对数似然成本  
 - dw  - 相对于w的损失梯度，因此与w相同的形状  
 - db  - 相对于b的损失梯度，因此与b的形状相同  

定义**优化函数**  
optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False)
此函数通过运行梯度下降算法来优化w和b
    
参数：  
 - w  - 权重，大小不等的数组（num_px * num_px * 3，1）  
 - b  - 偏差，一个标量  
 - X  - 维度为（num_px * num_px * 3，训练数据的数量）的数组。  
 - Y  - 真正的“标签”矢量（如果非猫则为0，如果是猫则为1），矩阵维度为(1,训练数据的数量)  
 - num_iterations  - 优化循环的迭代次数  
 - learning_rate  - 梯度下降更新规则的学习率  
 - print_cost  - 每100步打印一次损失值  

返回：  
 - params  - 包含权重w和偏差b的字典  
 - grads  - 包含权重和偏差相对于成本函数的梯度的字典  
 - 成本 - 优化期间计算的所有成本列表，将用于绘制学习曲线。  

提示：  
我们需要写下两个步骤并遍历它们：  
1）计算当前参数的成本和梯度，使用propagate（）。  
2）使用w和b的梯度下降法则更新参数。  

定义**预测函数**  
predict(w, b, X)  
使用学习逻辑回归参数 logistic(w，b) 预测标签是0还是1  

参数：  
 - w  - 权重，大小不等的数组（num_px * num_px * 3，1）  
 - b  - 偏差，一个标量  
 - X  - 维度为（num_px * num_px * 3，训练数据的数量）的数据  

返回：  
 - Y_prediction  - 包含X中所有图片的所有预测【0 | 1】的一个numpy数组（向量）  

定义**模型函数**  
model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False)  
通过调用之前实现的函数来构建逻辑回归模型

参数：
 - X_train  - numpy的数组,维度为（num_px * num_px * 3，m_train）的训练集
 - Y_train  - numpy的数组,维度为（1，m_train）（矢量）的训练标签集
 - X_test   - numpy的数组,维度为（num_px * num_px * 3，m_test）的测试集
 - Y_test   - numpy的数组,维度为（1，m_test）的（向量）的测试标签集
 - num_iterations  - 表示用于优化参数的迭代次数的超参数
 - learning_rate  - 表示optimize（）更新规则中使用的学习速率的超参数
 - print_cost  - 设置为true以每100次迭代打印成本

返回：
 - d  - 包含有关模型信息的字典。

### 2.4 绘制损失图像

In [ ]:
# import matplotlib.pyplot as plt
# costs = np.squeeze(d['costs'])
# # 从字典 d 中提取出一个名为 'costs' 的项
# # np.squeeze 移除数组中的单维度 确保 costs 是一个一维数组
# plt.plot(costs)
# plt.ylabel('cost')
# plt.xlabel('iterations (per hundreds)')
# plt.title("Learning rate =" + str(d["learning_rate"]))
# plt.show()

### 2.5 超参数的选择

**迭代次数**  
增加的迭代次数，训练集准确性提高了，但是测试集准确性却降低了。这称为过度拟合。  

**学习率**
学习率 $\alpha$ 决定了我们更新参数的速度。    
如果学习率过高，则成本可能会上下波动，甚至可能会发散，可能会“超过”最优值。
如果它太小，我们将需要太多迭代才能收敛到最佳值，当训练精度比测试精度高很多时，就会发生过拟合情况。

## week-3

### 3.1 W和b参数初始化

In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    """
    参数：
        n_x - 输入层节点的数量
        n_h - 隐藏层节点的数量
        n_y - 输出层节点的数量    
    返回：
        parameters - 包含参数的字典：
            W1 - 权重矩阵,维度为（n_h，n_x）
            b1 - 偏向量，维度为（n_h，1）
            W2 - 权重矩阵，维度为（n_y，n_h）
            b2 - 偏向量，维度为（n_y，1）
    """    
    
    W1 = np.random.randn(n_h,n_x) * 0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h) * 0.01
    b2 = np.zeros((n_y,1))

    # 使用断言确保我的数据格式是正确的
    assert(W1.shape == ( n_h , n_x ))
    assert(b1.shape == ( n_h , 1 ))
    assert(W2.shape == ( n_y , n_h ))
    assert(b2.shape == ( n_y , 1 ))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

### 3.2 前向传播函数

In [ ]:
def forward_propagation(X, parameters):
    """
    参数：
         X - 维度为（n_x，m）的输入数据。
         parameters - 初始化函数（initialize_parameters）的输出
    
    返回：
         A2 - 使用sigmoid()函数计算的第二次激活后的数值
         cache - 包含“Z1”，“A1”，“Z2”和“A2”的字典类型变量
     """
    
    # 从字典 “parameters” 中检索每个参数
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # 实现前向传播计算A2(概率)
    Z1 = np.dot(W1,X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = sigmoid(Z2)
    
    #使用断言确保我的数据格式是正确的
    assert(A2.shape == (1,X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

### 3.3 计算cost

$$
J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large{[} \small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right) \large{]}
$$ 

In [ ]:
def compute_cost(A2, Y):
    """
    计算方程中给出的交叉熵成本，
    
    参数：
         A2 - 使用sigmoid()函数计算的第二次激活后的数值
         Y - "True"标签向量,维度为（1，数量）         
    
    返回：
         成本 - 交叉熵成本
    """
    
    # 样本数量
    m = Y.shape[1] 

    # 计算交叉熵代价
    logprobs = Y*np.log(A2) + (1-Y)* np.log(1-A2)
    cost = -1/m * np.sum(logprobs)
    
    # 确保损失是我们期望的维度
    # 例如，turns [[17]] into 17 
    cost = np.squeeze(cost)     
                               
    assert(isinstance(cost, float))
    
    return cost

### 3.4 反向传播函数

In [ ]:
def backward_propagation(parameters, cache, X, Y):
    """
    使用上述说明搭建反向传播函数。
    
    参数：
     parameters - 包含我们的参数的一个字典类型的变量。
     cache - 包含“Z1”，“A1”，“Z2”和“A2”的字典类型的变量。
     X - 输入数据，维度为（2，数量）
     Y - “True”标签，维度为（1，数量）
    
    返回：
     grads - 包含W和b的导数一个字典类型的变量。
    """
    
    m = X.shape[1]
    
    # 首先，从字典“parameters”中检索W1和W2。
    W1 = parameters["W1"]
    W2 = parameters["W2"]
        
    # 还可以从字典“cache”中检索A1和A2。
    A1 = cache["A1"]
    A2 = cache["A2"]
    
    # 反向传播:计算 dW1、db1、dW2、db2。
    dZ2= A2 - Y
    dW2 = 1 / m * np.dot(dZ2,A1.T)
    db2 = 1 / m * np.sum(dZ2,axis=1,keepdims=True)
    dZ1 = np.dot(W2.T,dZ2) * (1-np.power(A1,2))
    dW1 = 1 / m * np.dot(dZ1,X.T)
    db1 = 1 / m * np.sum(dZ1,axis=1,keepdims=True)
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads 

### 3.5 更新参数

In [ ]:
def update_parameters(parameters, grads, learning_rate = 1.2):
    """
    使用上面给出的梯度下降更新规则更新参数
    
    参数：
     parameters - 包含参数的字典类型的变量。
     grads - 包含导数值的字典类型的变量。
     learning_rate - 学习速率
    
    返回：
     parameters - 包含更新参数的字典类型的变量。
    """
    
    # 从字典“parameters”中检索每个参数
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # 从字典“梯度”中检索每个梯度
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    # 每个参数的更新规则
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

### 3.6 整合

In [ ]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
    """
    参数：
        X - 数据集,维度为（2，示例数）
        Y - 标签，维度为（1，示例数）
        n_h - 隐藏层的数量
        num_iterations - 梯度下降循环中的迭代次数
        print_cost - 如果为True，则每1000次迭代打印一次成本数值
    
    返回：
        parameters - 模型学习的参数，它们可以用来进行预测。
     """    
    
    # 初始化参数，然后检索 W1, b1, W2, b2。输入:“n_x, n_h, n_y”。
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # 初始化参数，然后检索 W1, b1, W2, b2。
    # 输入:“n_x, n_h, n_y”。输出=“W1, b1, W2, b2，参数”。
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # 循环(梯度下降)
    for i in range(0, num_iterations):
         
        # 前项传播
        A2, cache = forward_propagation(X, parameters)
        
        # 计算成本
        cost = compute_cost(A2, Y, parameters)
        
        # 反向传播
        grads = backward_propagation(parameters, cache, X, Y)
        
        # 更新参数
        parameters = update_parameters(parameters, grads)
                  
        # 每1000次迭代打印成本
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

### 3.7 预测

In [ ]:
def predict(parameters, X): 
    """
    使用学习的参数，为X中的每个示例预测一个类
    
    参数：
        parameters - 包含参数的字典类型的变量。
        X - 输入数据（n_x，m）
    
    返回
        predictions - 我们模型预测的向量（红色：0 /蓝色：1）
     
     """
    
    # 使用前向传播计算概率，并使用 0.5 作为阈值将其分类为 0/1。
    A2, cache = forward_propagation(X, parameters)
    predictions = np.round(A2)
    
    return predictions

## week-4

### 4.1 搭建深层神经网络

与week-3类似